## TOC:
* [Parameters](#parameters)
* [Elliptic Curve Mathematics](#ec-math)
    * [Modular Inverse](#modular-inverse)
    * [Double](#double)
    * [Add](#add)
    * [Multiply](#multiply)
        * [Double-and-add algorithm](#double-add)
* [ECDSA](#ecdsa)
    * [Key Generation](#key-gen)
    * [Sign](#sign)
    * [Verify](#verity)


In [1]:
from IPython.display import clear_output
from time import sleep, time

***
## Parameters<a class="anchor" id="parameters"></a>

The following variables are the ones used in secp256k1, the elliptic curve used in Bitcoin.</br></br>
Elliptic curve fuction:   $y^2 = x^3 + ax + b$

In [2]:
a = 0
b = 7

# prime field
p = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 - 1

# order
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

G = [# genarator point
  55066263022277343669578718895168534326250603453777594175500187360389116729240,
  32670510020758816978083085130507043184471273380659243275938904335757337482424
]

***
## Elliptic Curve Mathematics<a class="anchor" id="ec-math"></a>


### Modular Inverse<a class="anchor" id="modular-inverse"></a>

Extended Euclidean algorithm

In [3]:
def inverse(a, m = p):
    m_orig = m
    if a == 0:
        raise ZeroDivisionError(f"Variable a={a}; should be bigger than 0.")
    a = a % m
    prevy, y = 0, 1
    while a > 1:
        q = m // a
        y, prevy = prevy - q * y, y
        a, m = m % a, a
    return y % m_orig

print(inverse(13, 47))
print(pow(13, -1, 47)) # pyhton3.8+ and above 

29
29


In [4]:
def animation(x, y, m):

    for i in range(x, x*y+1):
        print("Finite Fields: Multiplication")
        pos = i % m

        print(" "+" "*pos+str(pos))
        print("0"+" "*pos+"\u21E9"+" "*(m-pos-1)+str(m))
        print("|"+"\u2022"*m+"|")


        print("\n{} * {} = {}".format(x, y,str(pos)if i ==x*y else ""))
        sleep(.05)
        clear_output(wait=True)
    result = pos
    sleep(1.2)
    for i in range(result + 1 ,(result-1)*pow(x, -1, m)+2):
        print("Finite Fields: Multiplication (Find Inverse)")
        pos = i % m
        times = i // m

        print(" "+" "*pos+str(pos))
        print("0"+" "*pos+"\u21E9"+" "*(m-pos-1)+str(m))
        print("|"+"\u2022"*m+"|")


        print("\n{} * {} = {}".format(x, y, result))
        print("\n{} * {} = {}".format(result, times, i%m))
        sleep(.05)
        clear_output(wait=True)


#animation(8, 13, 47)

### Double<a class="anchor" id="double"></a>

Used in case the two points are the same. We calculate the tangent to the point and find out where else does intersect the curve.

$slope = (3x₁² + a) / 2y₁$</br>
$x = slope² - 2x₁$</br>
$y = slope * (x₁ - x) - y₁$

In [5]:
def double(point):
    slope = ((3 * point[0] ** 2 + a) * inverse((2 * point[1]), p)) % p
    x = (slope ** 2 - (2 * point[0])) % p
    y = (slope * (point[0] - x) - point[1]) % p
    return (x, y)

double(G)

(89565891926547004231252920425935692360644145829622209833684329913297188986597,
 12158399299693830322967808612713398636155367887041628176798871954788371653930)

### Add<a class="anchor" id="add"></a>

Used when the points are different.

$slope = (y₁ - y₂) / (x₁ - x₂)$</br>
$x = slope² - x₁ - x₂$</br>
$y = slope * (x₁ - x) - y₁$

In [6]:
def add(point1, point2):
  if point1 == point2:
    return double(point1)
  slope = ((point1[1] - point2[1]) * inverse(point1[0] - point2[0], p)) % p
  x = (slope ** 2 - point1[0] - point2[0]) % p
  y = ((slope * (point1[0] - x)) - point1[1]) % p
  return (x, y)

T = add(G, G)
print(T)
add(T, G)

(89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930)


(112711660439710606056748659173929673102114977341539408544630613555209775888121,
 25583027980570883691656905877401976406448868254816295069919888960541586679410)

### Multiply<a class="anchor" id="multiply"></a>
Used for sucesive additions.

In [131]:
def multiply(constant, point = G):
    acu = add(point, point)
    for _ in range(constant-2):
        acu = add(acu, point)
    return acu

start = time()
result = multiply(40000)
print(f"Execution time: {(time() - start)*1000} ms")
print(result)

Execution time: 2053.1883239746094 ms
(45144681238022915379838256640648974390095578890737782493692183842300544374804, 15493911184894630123643928078266102650003862558761004463646228136434723692345)


In [132]:
def uber_multiply(constant, point = G):
    current = point
    binary = bin(constant)[3:]
    for i in binary:
        current = double(current)
        if i == '1':
            current = add(current, point)
    return current

start = time()
result = uber_multiply(40000)
print(f"Execution time: {(time() - start)*1000} ms")
print(result)

Execution time: 1.0018348693847656 ms
(45144681238022915379838256640648974390095578890737782493692183842300544374804, 15493911184894630123643928078266102650003862558761004463646228136434723692345)


***
## ECDSA <a class="anchor" id="ecdsa"></a>

### Key Generation <a class="anchor" id="key-gen"></a>

The private key should be a random number less than orden 'n'.

In [143]:
d = 112757557418114203588093402336452206775565751179231977388358956335153294300646 # Pvivate key
Q = uber_multiply(d)

print("Public {}: {}".format("\U0001F511",Q))

Public 🔑: (33886286099813419182054595252042348742146950914608322024530631065951421850289, 9529752953487881233694078263953407116222499632359298014255097182349749987176)


### Sign<a class="anchor" id="sign"></a>